# SLR data reduction: under the hood

Here we disect the files (one ifproc and four roach) for an obsnum. Since an observation is typically one short **Cal** and one long **Map**, we have 10 files to process for an observation.

The **ifproc** samples at 125 Hz.  The roach boards at 10 Hz (the integration time). Each roach board has 4 beams, their timestamps are staggered, i.e. b0,b1,b2,b3,b0,b1,b2,b3,..... have incremental times at a cadence of about 0.025sec. Note that each of those are sampled independantly, thus they needs to be syncd by the software during reduction.

The information what kind of data the telescope is taking is in the **ifproc::BufPos** variable, and takes up four values:
* 0 = "on"
* 1 = "ref"
* 2 = "sky"
* 3 = "hot"

The **Cal** is two short bursts of "Hot" and "Sky".   The **Map** is many repetitions of "Ref" and "On". For the IRC benchmark data of 700" there are five "Ref" and four "On". Slewing is about 7-8".  Switching from "Cal" to "Map" is about 14".

In [ ]:
%matplotlib
import os
import numpy as np
import matplotlib.pyplot as pl

import netCDF4
import datetime

from lmtslr.spec import *
from lmtslr.ifproc import *
from lmtslr.viewer.spec_viewer import *
#from beam import *
#from beam_viewer import *
from lmtslr.utils.roach_file_utils import create_roach_list, lookup_roach_files
from lmtslr.utils.ifproc_file_utils import lookup_ifproc_file

from lmtslr.ifproc.ifproc import IFProcData, IFProcCal
from lmtslr.spec.spec import SpecBankCal, SpecBankData

from lmtslr.utils.reader import read_obsnum_ps

In [ ]:
# set root directory for the benchmark IRC_data
path = os.environ['DATA_LMT'] + '/../IRC_data'
       
# set obsnum for cal (0) and map (1)     
obsnum0 = 79447
obsnum1 = 79448

# get the filenames associated with these obsnums                                                                        
iffile0   = lookup_ifproc_file(obsnum0,path=path+'/ifproc')
rofile0,n = lookup_roach_files(obsnum0,path=path+'/spectrometer')

iffile1   = lookup_ifproc_file(obsnum1,path=path+'/ifproc')
rofile1,n = lookup_roach_files(obsnum1,path=path+'/spectrometer')


In [ ]:
print(path)
print(iffile0)
print(iffile1)

In [ ]:
def date_obs(date):
    """ convert to ISO """
    return datetime.datetime.fromtimestamp(date).strftime('%Y-%m-%dT%H:%M:%S')
    

In [ ]:
def get_ifproc(filename):
    """ get some header info from an ifproc file"""
    
    nc = netCDF4.Dataset(filename)
    
    bufpos = nc.variables['Data.TelescopeBackend.BufPos'][:]
    dates  = nc.variables['Data.TelescopeBackend.TelTime'][:]
    date0   = date_obs(dates[0])
    date1   = date_obs(dates[-1])
    tsky    = dates[-1] - dates[0]
    sname = filename[filename.rfind('/')+1:]
    
    cnt = []
    for b in [0,1,2,3]:
        idx = np.where(bufpos==b)
        cnt.append(len(idx[0]))
    print(sname,date0,date1,int(tsky),cnt)   
    
    nc.close()

In [ ]:
def get_roach(filename):
    """ get some roach board info """
    
    nc = netCDF4.Dataset(filename)
    
    dates = nc.variables['Data.Integrate.time'][:]
    date0   = date_obs(dates[0])
    date1   = date_obs(dates[-1])
    tsky    = dates[-1] - dates[0]
    sname   = filename[filename.rfind('/')+1:]
    nsky = len(dates)
    print(sname,date0,date1,int(tsky),nsky)  
    
    nc.close()

In [ ]:
get_ifproc(iffile0)
get_ifproc(iffile1)
for f in rofile0:
    get_roach(f)    
for f in rofile1:
    get_roach(f)

###    Detailed look at time

By plotting the time we can see slewing (for the Map, about 7-8") and switching hot/sky (for the Cal, about 2.5").

In [ ]:
def get_dates(filename,varname):
    nc = netCDF4.Dataset(filename)
    dates  = nc.variables[varname][:]
    nc.close()
    return dates

In [ ]:
if True:
    dates0 = get_dates(iffile0,'Data.TelescopeBackend.TelTime')
    dates1 = get_dates(iffile1,'Data.TelescopeBackend.TelTime')
else:
    dates0 = get_dates(rofile0[0],'Data.Integrate.time')
    dates1 = get_dates(rofile1[0],'Data.Integrate.time')   
dates2 = np.append(dates0,dates1)

    


In [ ]:
dates = dates2 - dates2[0]
pl.plot(range(len(dates)),dates);
print(dates[1]-dates[0])
print(dates[4]-dates[0])